In [1]:
!nvidia-smi

Thu Oct 12 05:06:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install --upgrade transformers


In [3]:
!pip install  transformers[setencespieces] dataset sacrebleu rouge_score py7zr -q

In [4]:
! pip install --upgrade accelerate
! pip install -y transformers accelerate
! pip install transformers accelerate



Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -y


In [5]:
# import os
# os.environ

In [6]:
!pip install datasets

In [7]:
from transformers import pipeline,set_seed
from datasets import load_dataset , load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_metric

from transformers  import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import  sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from transformers import AutoModelForSeq2SeqLM , AutoTokenizer
device = 'cuda' if torch.cuda.is_available() else "cpu"
device

'cuda'

In [9]:
# !pip install sentencepiece

In [10]:
!pip install transformers[sentencepiece]

In [11]:
model_ckpt= "google/pegasus-cnn_dailymail"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt,use_fast=False)
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("model_ckpt", use_fast=False)


model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Download & unzip the data

# !wget https://github.com/enth

In [13]:
from datasets import load_dataset

dataset_samsum = load_dataset("samsum")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [14]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [15]:
from huggingface_hub import dataset_info
dataset_info

<bound method HfApi.dataset_info of <huggingface_hub.hf_api.HfApi object at 0x7f2b08151e70>>

In [16]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum ]

print(f"Split length:{split_lengths}")
print(f"features:{dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary")

print(dataset_samsum['test'][1]["summary"])


Split length:[14732, 819, 818]
features:['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary
Eric and Rob are going to watch a stand-up on youtube.


In [17]:
def convert_examples_to_features(example_batch):
  input_encodings=tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)

  with tokenizer.as_target_tokenizer():
    target_encodings=tokenizer(example_batch['summary'],max_length=128,truncation=True)

  return{
      "input_ids":input_encodings['input_ids'],
      'attention_mask':input_encodings['attention_mask'],
      'labels':target_encodings['input_ids'],
  }

In [18]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features,batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

**Training**

In [19]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [20]:
from transformers import DataCollatorForSeq2Seq

seq2Seq_data_collactor=DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

In [26]:
from transformers import TrainingArguments , Trainer

trainer_args = TrainingArguments(
    output_dir="pegasus-samsum",num_train_epochs=1,warmup_steps=500,
    per_device_train_batch_size=1,per_device_eval_batch_size=1,
    weight_decay=0.01,logging_steps=10,
    evaluation_strategy='steps',eval_steps=500,save_steps=1e6,
    gradient_accumulation_steps=16

)

In [30]:
trainer= Trainer(model=model_pegasus,args=trainer_args,
                 tokenizer=tokenizer,data_collator=seq2Seq_data_collactor,
                 train_dataset=dataset_samsum_pt['test'],
                 eval_dataset=dataset_samsum_pt['validation'])

In [32]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.0765149733599495, metrics={'train_runtime': 171.3183, 'train_samples_per_second': 4.781, 'train_steps_per_second': 0.298, 'total_flos': 313317832187904.0, 'train_loss': 3.0765149733599495, 'epoch': 1.0})

Evaluation

In [35]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """

    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i + batch_size]


def calculate_metric_on_test_ds(dataset, metric ,model,tokenizer):
    """

    """

    article_batches= list(genrated_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(genrated_batch_sized_chunks(dataset[column_summary], batch_size))


    for article_batches,target_batches in tqdm(
        zip(article_batches,target_batches),total=len(article_batches)):

        inputs= tokenizer(article_batch,max_length=1024,truncation=True,
                          padding='max_length',return_tensors='pt')

        summaries=model.generate(input_ids=inputs['input_ids'].to(device),
                                 attention_mask=inputs[attention_mask].to(device),
                                 length_penalty=0.8,num_beams=8,max_length=128)

        decode_summaries =[tokenizer.decode(s,skip_special_tokens=True,clean_up_tokenization_spaces=True)
                          for s in summaries]

        decode_summaries =[d.replace(""," ")for d in decode_summaries ]

        metric.add_batch(predications=decode_summaries,references=target_batch)

    score=metric.compute()

    return score





In [38]:
rouge_names=["rouge1","rouge2","rougeL","rougeLsum"]
rouge_metric = load_metric=('rouge')
rouge_metric

'rouge'

In [ ]:
# score=calculate_metric_on_test_ds(
#     dataset_samsum['test'][0:10],rouge_metric,trainer.model,tokenizer,batch_size=2,column_text='dialouge',column_summary=''
# )

In [39]:
##save_model
model_pegasus.save_pretrained('pegasus-samsum-model')

In [40]:
#save tokenizer
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json')

In [42]:
tokenizer= AutoTokenizer.from_pretrained("/content/tokenizer")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [45]:
gen_kwargs={'length_penalty':0.8,'num_beams':8,'max_length':128}


sample_text=dataset_samsum['test'][0]['dialogue']

reference=dataset_samsum['test'][0]['summary']

pipe=pipeline('summarization',model='pegasus-samsum-model',tokenizer=tokenizer)

print("Dialouge")

print(sample_text)

print("\nReference Summary:")
print('refernce')

print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]['summary_text'])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialouge
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
refernce

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .


In [ ]:
from google.colab import drive
drive.mount('/content/drive')